In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv("~/PycharmProjects/Diabetes/finalNew.csv", sep=',')
SBP_data = data[['Patient_ID', 'PerformedDate', 'BP_res', 'Name_calc']]
SBP_data = SBP_data[SBP_data['Name_calc'] == 'sBP (mmHg)']
SBP_data = SBP_data.dropna()

SBP_data = SBP_data.sort_values(by='PerformedDate', ascending=True)

grouped_patients = SBP_data.groupby("Patient_ID")
grouped_lists = grouped_patients["BP_res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['BP_res'] = grouped_lists['BP_res'].apply(lambda x: list(set(x)))
data = grouped_lists[['BP_res']]
print(data)

            Patient_ID PerformedDate  BP_res   Name_calc
4863609  4000000368847    2019-03-04   139.0  sBP (mmHg)
4863610  4000000368847    2018-12-18   142.0  sBP (mmHg)
4863611  4000000368847    2016-01-18   146.0  sBP (mmHg)
4863612  4000000368357    2019-11-14   124.0  sBP (mmHg)
4863613  4000000368357    2015-11-23   121.0  sBP (mmHg)
...                ...           ...     ...         ...
7467650  5050180011096    2016-06-10   125.0  sBP (mmHg)
7467651  5050060019660    2017-04-25   113.0  sBP (mmHg)
7467652  5050060019660    2014-09-24   152.0  sBP (mmHg)
7467653  5050060019660    2015-04-14   154.0  sBP (mmHg)
7467654  5050070039110    2018-10-30   128.0  sBP (mmHg)

[2570192 rows x 4 columns]
                                                   BP_res
0       [100.0, 104.0, 108.0, 110.0, 115.0, 120.0, 90....
1       [161.0, 130.0, 131.0, 163.0, 132.0, 134.0, 136...
2       [128.0, 130.0, 132.0, 137.0, 138.0, 108.0, 110...
3       [128.0, 130.0, 132.0, 134.0, 136.0, 138.0, 140..

In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['BP_res'].at[index]) > 15 :
        seq = data['BP_res'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model.fit(X_train, y_train, epochs=200, verbose=0)
#for index, row in data.iterrows() :
# demonstrate prediction

In [ ]:
for index, row in data.iterrows() :
    if len(data['BP_res'].at[index]) > 15 :
        seq = data['BP_res'].at[index]
        #x_input = np.asarray(X_test[-4:-1])
        #x_input = X_test.reshape((1, n_steps, n_features))
        yhat = model.predict(X_test, verbose=0)
        y_actual = np.asarray(y_test[-1:])
        model.evaluate(X_train, y_train)